In [ ]:
# run_sagemaker_inference.py (boto3 업로드 버전)
import os
import boto3
import sagemaker
from sagemaker.pytorch import PyTorch
from datetime import datetime

# ===== 사용자 환경 설정 =====
S3_BUCKET_NAME = "say1-5team-bucket"
S3_PROJECT_PREFIX = "yolo-final-skin-job"

# ===== SageMaker 세션 & 역할 =====
sagemaker_session = sagemaker.Session()  # default_bucket 지정하지 않음(불필요한 S3 테스트 방지)
role = sagemaker.get_execution_role()
print(f"사용 S3 버킷: s3://{S3_BUCKET_NAME}")
print(f"사용 IAM 역할: {role}")

# ===== 로컬 경로 =====
local_data_path = './data'        # 추론 대상 이미지 폴더
local_model_path = './best.pt'    # YOLO 학습 완료된 모델
local_script_path = './inference.py'  # entry_point


In [19]:
# ===== boto3로 업로드 =====
s3 = boto3.client('s3')

import os, hashlib, boto3
from botocore.exceptions import ClientError

s3 = boto3.client("s3")

# ===== 설정 =====
S3_BUCKET_NAME = "say1-5team-bucket"
S3_PROJECT_PREFIX = "yolo-final-skin-job"

data_prefix  = f"{S3_PROJECT_PREFIX}/input/data"
model_prefix = f"{S3_PROJECT_PREFIX}/input/model"

# ✅ 이미 업로드 되어 있다고 가정 → 업로드 함수 호출 주석 처리
# sync_dir_to_s3("./data", S3_BUCKET_NAME, data_prefix, force=False, compare_etag=True)
# upload_file_if_needed("./best.pt", S3_BUCKET_NAME, f"{model_prefix}/best.pt", force=False, compare_etag=True)

# ===== S3 URI 직접 지정 =====
s3_input_data_uri  = f"s3://{S3_BUCKET_NAME}/{data_prefix}"
s3_input_model_uri = f"s3://{S3_BUCKET_NAME}/{model_prefix}"

print("📌 데이터 S3 URI:", s3_input_data_uri)
print("📌 모델   S3 URI:", s3_input_model_uri)



📌 데이터 S3 URI: s3://say1-5team-bucket/yolo-final-skin-job/input/data
📌 모델   S3 URI: s3://say1-5team-bucket/yolo-final-skin-job/input/model


In [ ]:
# ===== requirements.txt 생성 =====
with open("requirements.txt", "w") as f:
    f.write("ultralytics\n")
    f.write("opencv-python\n")

# ===== Estimator 설정 =====
estimator = PyTorch(
    entry_point=os.path.basename(local_script_path),
    source_dir='.',
    role=role,
    instance_count=1,
    instance_type='ml.g4dn.2xlarge',
    framework_version='2.0.0',
    py_version='py310',
    sagemaker_session=sagemaker_session,
    output_path=f"s3://{S3_BUCKET_NAME}/{S3_PROJECT_PREFIX}/output",
    requirements_file="requirements.txt",
    tags=[{"Key": "project", "Value": "pre-5team"}],
)

# ===== 채널 매핑 =====
# inference.py에서 /opt/ml/input/data/training, /opt/ml/input/data/model 기준으로 읽도록 작성되어 있어야 함
inputs = {
    "training": s3_input_data_uri,
    "model":    s3_input_model_uri
}

# ===== 실행 =====
training_job_name = f"aws-say1-5team-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
print(f"SageMaker 추론 작업 시작: {training_job_name}")
estimator.fit(inputs=inputs, job_name=training_job_name)

print(f"작업 완료! 결과는 s3://{S3_BUCKET_NAME}/{S3_PROJECT_PREFIX}/output 에 저장됨")

# ===== 로그 스트리밍 =====
sess = sagemaker.Session()
sess.logs_for_job(job_name=training_job_name, wait=True)
